<div style="font-family:'times new roman'; background-color:#093145; padding:35px; text-align:center; color:white">
    <h1>InMemDB</h1>
    <p>InMemDB is a python package that allows you to create a sqlite relational database in memory
     out of a pandas.DataFrame object or any array-like sequence type.<p>
</div>
<div style="font-family:'times new roman'; background-color:#f5f5f5; padding:30px; text-align:center; color:#093145">
    <h1>Advantages</h1><br>
    <div style="text-align:left; margin:auto; max-width:500px;">
        <ul>
            <li>Combines SQLilte with the best data manipulation and analysis tool Pandas</li>
            <li>Can be used to Extract, Transform, and Load data</li>
            <li>Those who are more familar with SQL can still manipulate the powerful pandas DataFrame</li>
        </ul>
    </div>
</div>

In [220]:
from InMemDB import InMemDB
import pandas as pd
import os
import re

In [221]:
db = InMemDB()

In [ ]:
os.chdir(r'lego-database')

In [223]:
tables = os.listdir()
tables

['colors.csv',
 'inventories.csv',
 'inventory_parts.csv',
 'inventory_sets.csv',
 'parts.csv',
 'part_categories.csv',
 'sets.csv',
 'themes.csv']

In [224]:
def name(i):
    return re.findall('[A-Za-z_A-Za-z]+',i)[0]

In [225]:
name('inventory_parts.csv')

'inventory_parts'

In [226]:
for i in tables:
    df = pd.read_csv(i)
    db.CreateTableDF(table_name=name(i),df=df)

In [227]:
db.TableNames()

['colors',
 'inventories',
 'inventory_parts',
 'inventory_sets',
 'part_categories',
 'parts',
 'sets',
 'themes']

In [228]:
q1='''
SELECT * FROM inventories;
'''

In [229]:
q2='''
SELECT * FROM inventory_parts;
'''

In [230]:
q3='''
SELECT * FROM sets;
'''

In [231]:
db.query(q1).head(3)

,id,version,set_num
0,1,1,7922-1
1,3,1,3931-1
2,4,1,6942-1


In [232]:
db.query(q2).head(3)

,inventory_id,part_num,color_id,quantity,is_spare
0,1,48379c01,72,1,f
1,1,48395,7,1,f
2,1,mcsport6,25,1,f


In [233]:
db.query(q3).head(3)

,set_num,name,year,theme_id,num_parts
0,00-1,Weetabix Castle,1970,414,471
1,0011-2,Town Mini-Figures,1978,84,12
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,2


In [234]:
q4 = '''
SELECT i.id,
       s.name,
       i.version,
       i.set_num,
       ip.part_num,
       ip.color_id,
       ip.quantity,
       ip.is_spare

FROM inventories i
LEFT JOIN inventory_parts ip
ON i.id = ip.inventory_id
LEFT JOIN sets s
ON i.set_num = s.set_num
WHERE name = 'Butterfly Girl';
'''

In [242]:
df = db.query(q4)
df

,id,name,version,set_num,part_num,color_id,quantity,is_spare
0,18645,Butterfly Girl,1,71018-7,15427pr0173,226,1,f
1,18645,Butterfly Girl,1,71018-7,33647,45,1,f
2,18645,Butterfly Girl,1,71018-7,3626cpr2202,14,1,f
3,18645,Butterfly Girl,1,71018-7,3742,5,1,t
4,18645,Butterfly Girl,1,71018-7,3742,5,3,f
5,18645,Butterfly Girl,1,71018-7,41879bpr0010,29,1,f
6,18645,Butterfly Girl,1,71018-7,88646,0,1,f
7,18645,Butterfly Girl,1,71018-7,973pr3833c01,212,1,f
8,18645,Butterfly Girl,1,71018-7,99249,27,1,f


In [243]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 8 columns):
id          9 non-null int64
name        9 non-null object
version     9 non-null int64
set_num     9 non-null object
part_num    9 non-null object
color_id    9 non-null int64
quantity    9 non-null int64
is_spare    9 non-null object
dtypes: int64(4), object(4)
memory usage: 704.0+ bytes


In [238]:
q5 = '''
SELECT i.id,
       s.name,
       i.version,
       i.set_num,
       ip.part_num,
       ip.color_id,
       ip.quantity,
       ip.is_spare

FROM inventories i
LEFT JOIN inventory_parts ip
ON i.id = ip.inventory_id
LEFT JOIN sets s
ON i.set_num = s.set_num

'''

In [244]:
db.query(q5).head(5)

,id,name,version,set_num,part_num,color_id,quantity,is_spare
0,1,McDonald's Sports Set Number 6 - Orange Vest S...,1,7922-1,48379c01,72.0,1.0,f
1,1,McDonald's Sports Set Number 6 - Orange Vest S...,1,7922-1,48395,7.0,1.0,f
2,1,McDonald's Sports Set Number 6 - Orange Vest S...,1,7922-1,mcsport6,25.0,1.0,f
3,1,McDonald's Sports Set Number 6 - Orange Vest S...,1,7922-1,paddle,0.0,1.0,f
4,3,Emma's Splash Pool,1,3931-1,11816pr0005,78.0,1.0,f


In [240]:
q6 = '''
SELECT name,
       set_num,
       sum(quantity) AS quantity
FROM
(
    SELECT i.id,
           s.name AS name,
           i.version,
           i.set_num AS set_num,
           ip.part_num,
           ip.color_id,
           ip.quantity AS quantity,
           ip.is_spare

    FROM inventories i
    LEFT JOIN inventory_parts ip
    ON i.id = ip.inventory_id
    LEFT JOIN sets s
    ON i.set_num = s.set_num
)
WHERE quantity IS NOT NULL
GROUP BY name, set_num
ORDER BY quantity DESC

'''

In [245]:
db.query(q6).head(20)

,name,set_num,quantity
0,Taj Mahal,10189-1,5962
1,Star Wars / M&M Mosaic - Promo Set,SWMP-1,5462
2,Millennium Falcon - UCS,10179-1,5217
3,Window Exploration Bag,2000409-1,5200
4,Firehouse Headquarters,75827-1,4788
5,Personalised Mosaic Portrait,40179-1,4502
6,Tower Bridge,10214-1,4335
7,Big Ben,10253-1,4259
8,The Disney Castle,71040-1,4194
9,Assembly Square,10255-1,4147
